#Run tflite inference on a example Dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tflite_runtime.interpreter as tflite


ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)



: 

: 

pip install tensorflow-addons

In [8]:
interpreter = tflite.Interpreter('/media/psf/Home/MLProjects/ASL/holistic/model.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")
#print(found_signatures)
train = pd.read_csv('/media/psf/Home/MLProjects/ASL/asl-signs/train.csv')
# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

#pq_file = '/media/psf/Home/Downloads/ASL/asl-signs/train_landmark_files/16069/10042041.parquet

In [9]:
def get_prediction(prediction_fn, pq_file):
    xyz_np = load_relevant_data_subset(pq_file)
    prediction = prediction_fn(inputs=xyz_np)
    pred = prediction['outputs'].argmax()
    sign = ORD2SIGN[pred]
    pred_conf = prediction['outputs'][pred]
    print(f'PREDICTED SIGN:{sign} [{pred}], CONFIDENCE:{pred_conf:0.4}')

In [10]:
get_prediction(prediction_fn,'test23.parquet')

PREDICTED SIGN:duck [67], CONFIDENCE:0.3046


In [11]:
train.query('sequence_id == 10042041')

,path,participant_id,sequence_id,sign,sign_ord
101,train_landmark_files/16069/10042041.parquet,16069,10042041,green,102


In [12]:
train['sign'].unique()

array(['blow', 'wait', 'cloud', 'bird', 'owie', 'duck', 'minemy', 'lips',
       'flower', 'time', 'vacuum', 'apple', 'puzzle', 'mitten', 'there',
       'dry', 'shirt', 'owl', 'yellow', 'not', 'zipper', 'clean',
       'closet', 'quiet', 'have', 'brother', 'clown', 'cheek', 'cute',
       'store', 'shoe', 'wet', 'see', 'empty', 'fall', 'balloon',
       'frenchfries', 'finger', 'same', 'cry', 'hungry', 'orange', 'milk',
       'go', 'drawer', 'TV', 'another', 'giraffe', 'wake', 'bee', 'bad',
       'can', 'say', 'callonphone', 'finish', 'old', 'backyard', 'sick',
       'look', 'that', 'black', 'yourself', 'open', 'alligator', 'moon',
       'find', 'pizza', 'shhh', 'fast', 'jacket', 'scissors', 'now',
       'man', 'sticky', 'jump', 'sleep', 'sun', 'first', 'grass', 'uncle',
       'fish', 'cowboy', 'snow', 'dryer', 'green', 'bug', 'nap', 'feet',
       'yucky', 'morning', 'sad', 'face', 'penny', 'gift', 'night',
       'hair', 'who', 'think', 'brown', 'mad', 'bed', 'drink', 'stay',
